# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Arxiv has 55 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.17694


extracting tarball to tmp_2310.17694...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.17694/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2310.17694/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

✔ → 31031:\section{Results} \label{sec:results}
  ↳ 42895:\section{Discussion} \label{sec:discussion}


✔ → 42895:\section{Discussion} \label{sec:discussion}
  ↳ 55270:\section{Summary \& Conclusions} \label{sec:conclusions}
✔ → 55270:\section{Summary \& Conclusions} \label{sec:conclusions}
  ↳ 57658:\section{A comparison between the observed and modeled Lick indices} \label{ap:A}
✔ → 57658:\section{A comparison between the observed and modeled Lick indices} \label{ap:A}
  ↳ 59280:\section{Mock analysis} \label{ap:B}
✔ → 59280:\section{Mock analysis} \label{ap:B}
  ↳ 61189:end


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


Found 89 bibliographic references in tmp_2310.18000/main.bbl.
syntax error in line 263: '=' expected


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Gallazzi} {et~al.}({in prep.}){Gallazzi}, {Zibetti}, {Kaushal}, {Nersesian}, {van der Wel},  {the LEGAC Team}}]{Gallazzi_2023ApJ}{Gallazzi}, A., {Zibetti}, S., {Kaushal}, Y., {et~al.} {in prep.}, \apj
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.17694-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.17694) | **Where do stars explode in the ISM? -- The distribution of dense gas  around massive stars and supernova remnants in M33**  |
|| S. K. Sarbadhicary, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2023-10-30*|
|*Comments*| *34 pages, 14 figures. Submitted to ApJ. Comments welcome! The density distributions will be made publicly available after journal acceptance of manuscript. Please feel free to contact us in the meantime if you would like to use them*|
|**Abstract**| Star formation in galaxies is regulated by turbulence, outflows, gas heating and cloud dispersal -- processes which depend sensitively on the properties of the interstellar medium (ISM) into which supernovae (SNe) explode. Unfortunately, direct measurements of ISM environments around SNe remain scarce, as SNe are rare and often distant. Here we demonstrate a new approach: mapping the ISM around the massive stars that are soon to explode. This provides a much larger census of explosion sites than possible with only SNe, and allows comparison with sensitive, high-resolution maps of the atomic and molecular gas from the Jansky VLA and ALMA. In the well-resolved Local Group spiral M33, we specifically observe the environments of red supergiants (RSGs, progenitors of Type II SNe), Wolf-Rayet stars (WRs, tracing stars $>$30 M$_{\odot}$, and possibly future stripped-envelope SNe), and supernova remnants (SNRs, locations where SNe have exploded). We find that massive stars evolve not only in dense, molecular-dominated gas (with younger stars in denser gas), but also a substantial fraction ($\sim$45\% of WRs; higher for RSGs) evolve in lower-density, atomic-gas-dominated, inter-cloud media. We show that these measurements are consistent with expectations from different stellar-age tracer maps, and can be useful for validating SN feedback models in numerical simulations of galaxies. Along with the discovery of a 20-pc diameter molecular gas cavity around a WR, these findings re-emphasize the importance of pre-SN/correlated-SN feedback evacuating the dense gas around massive stars before explosion, and the need for high-resolution (down to pc-scale) surveys of the multi-phase ISM in nearby galaxies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.18000-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.18000) | **Less is less: photometry alone cannot predict the observed spectral  indices of $z\sim1$ galaxies from the LEGA-C spectroscopic survey**  |
|| A. Nersesian, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-10-30*|
|*Comments*| *13 pages, 8 figures, accepted 26 October 2023*|
|**Abstract**| We test whether we can predict optical spectra from deep-field photometry of distant galaxies. Our goal is to perform a comparison in data space, highlighting the differences between predicted and observed spectra. The Large Early Galaxy Astrophysics Census (LEGA-C) provides high-quality optical spectra of thousands of galaxies at redshift $0.6<z<1$. Broad-band photometry of the same galaxies, drawn from the recent COSMOS2020 catalog, is used to predict the optical spectra with the spectral energy distribution (SED) fitting code Prospector and the MILES stellar library. The observed and predicted spectra are compared in terms of two age and metallicity-sensitive absorption features (H$\delta_\mathrm{A}$ and Fe4383). The global bimodality of star-forming and quiescent galaxies in photometric space is recovered with the model spectra. But the presence of a systematic offset in the Fe4383 line strength and the weak correlation between the observed and modeled line strength imply that accurate age or metallicity determinations cannot be inferred from photometry alone. For now we caution that photometry-based estimates of stellar population properties are determined mostly by the modeling approach and not the physical properties of galaxies, even when using the highest-quality photometric datasets and state-of-the-art fitting techniques. When exploring a new physical parameter space (i.e. redshift or galaxy mass) high-quality spectroscopy is always needed to inform the analysis of photometry. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.17751-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.17751) | **Global Chemical Transport on Hot Jupiters: Insights from 2D VULCAN  photochemical model**  |
|| S.-M. Tsai, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-10-30*|
|*Comments*| *18 pages, 20 figures, submitted to ApJ*|
|**Abstract**| The atmospheric dynamics of tidally-locked hot Jupiters is dominated by the equatorial winds. Understanding the interaction between global circulation and chemistry is crucial in atmospheric studies and interpreting observations. Two-dimensional (2D) photochemical transport models shed light on how the atmospheric composition depends on circulation. In this paper, we introduce the 2D photochemical transport model, VULCAN 2D, which improves on the pseudo-2D approaches by allowing for non-uniform zonal winds. We extensively validate our VULCAN 2D with analytical solutions and benchmark comparisons. Applications to HD 189733 b and HD 209458 b reveal distinct characteristics in horizontal transport-dominated and vertical mixing-dominated regimes. Motivated by the inferred carbon-rich atmosphere by Giacobbe et al. (2021), we find that HD 209458 b with super-solar carbon-to-oxygen ratio (C/O) exhibits pronounced C2H4 absorption on the morning limb but not on the evening limb, owing to horizontal transport from the nightside. We discuss when a pseudo-2D approach is a valid assumption and its inherent limitations. Finally, we demonstrate the effect of horizontal transport in transmission observations and its impact on the morning-evening limb asymmetry with synthetic spectra, highlighting the need to consider global transport when interpreting exoplanet atmospheres. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.17694.md
    + _build/html/tmp_2310.17694/./4paneldensities.png
    + _build/html/tmp_2310.17694/./Fig_M33_zoomed.png
    + _build/html/tmp_2310.17694/./mockpopulations_4panel.png
exported in  _build/html/2310.18000.md
    + _build/html/tmp_2310.18000/./strict_fig2_alt4_crop.png
    + _build/html/tmp_2310.18000/./strict_fig3_alt4_miles_mist_crop_wn3.png
    + _build/html/tmp_2310.18000/./chi2_distribution.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Mdot}{M_{\odot} yr^{-1}\xspace}$
$\newcommand{\kms}{km s^{-1}\xspace}$
$\newcommand{\Msunpc}{M_{\odot} pc^{-2}\xspace}$
$\newcommand{\fmol}{f_{\mathrm{H_2}}\xspace}$
$\newcommand{\hi}{\ion{H}{1}\xspace}$
$\newcommand{\hii}{\ion{H}{2}\xspace}$
$\newcommand{\cmc}{cm^{-3}\xspace}$
$\newcommand{\mum}{\mum\xspace}$
$\newcommand{\cmipsunit}{(erg s^{-1})(M_{\odot} yr^{-1})^{-1}\xspace}$</div>



<div id="title">

# Where do stars explode in the ISM? -- The distribution of dense gas around massive stars and supernova remnants in M33

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.17694-b31b1b.svg)](https://arxiv.org/abs/2310.17694)<mark>Appeared on: 2023-10-30</mark> -  _34 pages, 14 figures. Submitted to ApJ. Comments welcome! The density distributions will be made publicly available after journal acceptance of manuscript. Please feel free to contact us in the meantime if you would like to use them_

</div>
<div id="authors">

S. K. Sarbadhicary, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Star formation in galaxies is regulated by turbulence, outflows, gas heating and cloud dispersal -- processes which depend sensitively on the properties of the interstellar medium (ISM) into which supernovae (SNe) explode. Unfortunately, direct measurements of ISM environments around SNe remain scarce, as SNe are rare and often distant. Here we demonstrate a new approach: mapping the ISM around the massive stars that are soon to explode. This provides a much larger census of explosion sites than possible with only SNe, and allows comparison with sensitive, high-resolution maps of the atomic and molecular gas from the Jansky VLA and ALMA. In the well-resolved Local Group spiral M33, we specifically observe the environments of red supergiants (RSGs, progenitors of Type II SNe), Wolf-Rayet stars (WRs, tracing stars $>$ 30 $\Msun$ , and possibly future stripped-envelope SNe), and supernova remnants (SNRs, locations where SNe have exploded). We find that massive stars evolve not only in dense, molecular-dominated gas (with younger stars in denser gas), but also a substantial fraction ( $\sim$ 45 \% of WRs; higher for RSGs) evolve in lower-density, atomic-gas-dominated, inter-cloud media. We show that these measurements are consistent with expectations from different stellar-age tracer maps, and can be useful for validating SN feedback models in numerical simulations of galaxies. Along with the discovery of a 20-pc diameter molecular gas cavity around a WR, these findings re-emphasize the importance of pre-SN/correlated-SN feedback evacuating the dense gas around massive stars before explosion, and the need for high-resolution (down to pc-scale) surveys of the multi-phase ISM in nearby galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2310.17694/./4paneldensities.png" alt="Fig8" width="100%"/>

**Figure 8. -** Cumulative distribution functions showing $H_2$, $\hi$, and total ($H_2$ + $\hi$) surface densities. Distributions are shown for the locations of WRs (blue lines), SNRs (green lines) and RSGs (red lines), while grey histograms denote the distribution of densities of the full M33 region, representing what would be expected for a random distribution. Vertical dashed lines denote the completeness limit of each dataset. Colored circles show the median of the distribution above the completeness limit. The distributions show a higher probability of more massive stars (e.g. the WRs) to evolve in denser gas. (*fig:3paneldensities*)

</div>
<div id="div_fig2">

<img src="tmp_2310.17694/./Fig_M33_zoomed.png" alt="Fig7" width="100%"/>

**Figure 7. -** The spatial distribution of red supergiants (RSGs), Wolf-Rayet stars (WRs) and supernova remnants (SNRs) with respect to the cold ($H_2$ + $\hi$) ISM in the M33 ACA survey region (shown in the inset plot). Orange circles represent RSGs with log $(L/L_{\odot})\geq 4.7$(roughly corresponding to $\gtrsim$12$\Msun$). Bluish circles show WRs and green stars show SNRs. $\hi$ is shown in grey-scale with the density range shown in the bottom colorbar. Contours of CO (2-1) emission (converted to $H_2$ surface densities) are also shown at levels of 1.9, 7.6, 15.3 and 30.6 M$_{\odot}$ pc$^{-2}$. The prominent star-forming regions in the ACA footprint -- NGC 604, IC 142 and NGC 595 -- are specifically labelled (showing high WR concentrations). The black box shows a region containing a WR and SNR with higher-resolution ALMA data in Figure 6. The map shows the 50 pc-scale cold ISM environments where core-collapse SNe will occur in the future (or have occured in the case of SNRs). Statistical analysis of these environments and their implications are discussed in Section \ref{sec:results} and \ref{sec:discussion}. (*fig:maps*)

</div>
<div id="div_fig3">

<img src="tmp_2310.17694/./mockpopulations_4panel.png" alt="Fig11" width="100%"/>

**Figure 11. -** Cumulative density histograms of the observed and modeled populations of RSGs, WRs, and SNRs from Section \ref{sec:sfrtracers}. Dark grey histogram in each panel shows the bulk $H_2$ distribution in M33 in the ACA area. Solid colored line denotes the $H_2$ surface density of the star category. Colored shaded region shows the 5th-95th percentile region of the mock populations drawn according to Section \ref{sec:sfrtracers}.  (*fig:mocks*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.17694"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\vsig}{v_{5}/\sigma_{0}}$
$\newcommand{\rA}{\mathrm{Å}}$
$\newcommand{\msun}{\mathrm{M}_{\sun}}$
$\newcommand{\logm}{\mathrm{logM/\msun}}$
$\newcommand{\sigs}{\sigma'_{\star,\rm{int}}}$
$\newcommand{\sigg}{\sigma'_{g,\rm{int}}}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hg}{H\gamma}$
$\newcommand{\hd}{H\delta}$
$\newcommand{\dfn}{D4000_n}$
$\newcommand{\hda}{\hd_\mathrm{A}}$
$\newcommand{\mgt}{\mathrm{Mg_{2}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand{\e}{é}$
$\newcommand{\amen}[1]{\textbf{\textit{#1}}}$
$\newcommand{\arraystretch}{2.0}$</div>



<div id="title">

# Less is less: photometry alone cannot predict the observed spectral indices of $z\sim1$ galaxies from the LEGA-C spectroscopic survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.18000-b31b1b.svg)](https://arxiv.org/abs/2310.18000)<mark>Appeared on: 2023-10-30</mark> -  _13 pages, 8 figures, accepted 26 October 2023_

</div>
<div id="authors">

A. Nersesian, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** We test whether we can predict optical spectra from deep-field photometry of distant galaxies. Our goal is to perform a comparison in data space, highlighting the differences between predicted and observed spectra. The Large Early Galaxy Astrophysics Census (LEGA-C) provides high-quality optical spectra of thousands of galaxies at redshift $0.6<z<1$. Broad-band photometry of the same galaxies, drawn from the recent COSMOS2020 catalog, is used to predict the optical spectra with the spectral energy distribution (SED) fitting code Prospector and the MILES stellar library. The observed and predicted spectra are compared in terms of two age and metallicity-sensitive absorption features (H$\delta_\mathrm{A}$ and Fe4383). The global bimodality of star-forming and quiescent galaxies in photometric space is recovered with the model spectra. But the presence of a systematic offset in the Fe4383 line strength and the weak correlation between the observed and modeled line strength imply that accurate age or metallicity determinations cannot be inferred from photometry alone. For now we caution that photometry-based estimates of stellar population properties are determined mostly by the modeling approach and not the physical properties of galaxies, even when using the highest-quality photometric datasets and state-of-the-art fitting techniques. When exploring a new physical parameter space (i.e. redshift or galaxy mass) high-quality spectroscopy is always needed to inform the analysis of photometry. 

</div>

<div id="div_fig1">

<img src="tmp_2310.18000/./strict_fig2_alt4_crop.png" alt="Fig3" width="100%"/>

**Figure 3. -** Predicted $\hd$a and Fe4383 absorption features from {\tt Prospector} fits to the COSMOS2020 photometry compared to observations. Galaxies are color-coded by their $UVJ$--diagram classification to star-forming (blue diamonds) and quenched (red points). The absorption-only models are compared to the observed values (corrected for emission). The black line shows the one-to-one relation. The various markers correspond to the star-forming (orange) and quenched (green) galaxies shown in Fig. \ref{fig:sed_examples}. The statistics of the mean offset and the Spearman's rank correlation coefficient ($\rho$), are shown in the top-left corner of each panel. (*fig:hd_fe4383_obs_vs_prd*)

</div>
<div id="div_fig2">

<img src="tmp_2310.18000/./strict_fig3_alt4_miles_mist_crop_wn3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Left: Observed $\hd$a versus Fe4383. Middle: Predicted $\hd$a versus Fe4383. Right: Perturbed model values according to the individual observed errors. Galaxies are color-coded with $\log \sigma_\star$ from the LEGA-C DR3 catalog. The SSP model grid is shown with the red (Age) and blue (metallicity) lines. The statistics of the scatter and the Spearman's rank correlation coefficient ($\rho$) are shown in the bottom-left corner of each panel. The median uncertainties of each axis are shown in the top-right corner of each panel. (*fig:hd_vs_fe4383*)

</div>
<div id="div_fig3">

<img src="tmp_2310.18000/./chi2_distribution.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of the reduced $\chi^2$ between the predicted spectra with {\tt Prospector} and the observed LEGA-C spectra. The Kernel Density Estimate (KDE) distribution is shown in red, while the solid black line shows the median value. The dotted and dashed black lines indicate the $16^\mathrm{th}$ and $84^\mathrm{th}$ percentiles of the distribution, respectively. (*fig:chisqr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.18000"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

309  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
